Adding rules to KG

In [7]:
import pandas as pd

# Define your rules
rules = {
    "DGIDB::ANTAGONIST::Gene:Compound": "DGIDB::ALLOSTERIC MODULATOR::Gene:Compound",
    "DGIDB::POSITIVE ALLOSTERIC MODULATOR::Gene:Compound": "DGIDB::ALLOSTERIC MODULATOR::Gene:Compound",
    # Add all other rules
}

# Read the triples from the file
df = pd.read_csv('drkg.tsv', sep='\t', header=None, names=['head', 'relation', 'tail'])

# Apply the rules to generate new triples
new_triples = []
for idx, row in df.iterrows():
    if row['relation'] in rules:
        new_relation = rules[row['relation']]
        new_triples.append([row['head'], new_relation, row['tail']])

# Convert the list of new triples to a DataFrame and concatenate it with the original DataFrame
new_triples_df = pd.DataFrame(new_triples, columns=['head', 'relation', 'tail'])
df = pd.concat([df, new_triples_df])




TransE + Rules

In [8]:
import torch
from pykeen.models import TransE
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline

# Create a triples factory from the DataFrame
triples_factory = TriplesFactory.from_labeled_triples(df.values)

# Train the model using the stochastic local closed world assumption training approach
result = pipeline(
    model=TransE,
    training=triples_factory,
    testing=triples_factory,  # Here we're testing on the training set, but you should split your data
    model_kwargs=dict(embedding_dim=50),  # Choose this carefully
    training_kwargs=dict(num_epochs=100),  # Choose this carefully
    random_seed=1234,
    device='cuda',  # use 'cuda' for GPU
)

# After training, the model's embeddings can be accessed with
model = result.model
entity_embeddings = model.entity_embeddings()
relation_embeddings = model.relation_embeddings()

INFO:pykeen.pipeline.api:Using device: cuda
Training epochs on cpu:  39%|███▉      | 39/100 [2:54:37<4:04:43, 240.72s/epoch, loss=0.0411, prev_loss=0.0412]

TransE (no rules)

In [ ]:
# Read the triples from the file
df = pd.read_csv('drkg.tsv', sep='\t', header=None, names=['head', 'relation', 'tail'])

# Create a triples factory from the DataFrame
triples_factory = TriplesFactory.from_labeled_triples(df.values)

# Train the model using the stochastic local closed world assumption training approach
result = pipeline(
    model=TransE,
    training=triples_factory,
    testing=triples_factory,  # Here we're testing on the training set, but you should split your data
    model_kwargs=dict(embedding_dim=50),  # Choose this carefully
    training_kwargs=dict(num_epochs=100),  # Choose this carefully
    random_seed=1234,
    device='cuda',  # use 'cuda' for GPU
)

# After training, the model's embeddings can be accessed with
model_B = result.model
entity_embeddings_B = model.entity_embeddings()
relation_embeddings_B = model.relation_embeddings()

INFO:pykeen.pipeline.api:Using device: cuda
Training epochs on cpu:   0%|          | 0/100 [00:31<?, ?epoch/s]


KeyboardInterrupt: 